# 4. 작사가 인공지능 만들기

**LSTM 모델과 셰익스피어 데이터셋을 사용해 간단한 작사가 인공지능을 만들어 본다.**

---

## 목차
- Step 1. 데이터 다운로드
- Step 2. 데이터 읽어오기
  - glob 모듈 사용하여 파일 읽기
- Step 3. 데이터 정제
  - 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거
  - => 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기
- Step 4. 평가 데이터셋 분리
  - 훈련 데이터와 평가 데이터를 분리
  - 단어장의 크기는 12,000 이상, 총 데이터의 20% 를 평가 데이터셋으로 사용
- Step 5. 인공지능 만들기
  - 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)
  - 모델이 생성한 가사 한 줄 확인
- 회고

---

## 4-7. 프로젝트: 멋진 작사가 만들기

### Step 1. 데이터 다운로드
/lyricist/data/lyrics에 데이터가 있습니다.

### Step 2. 데이터 읽어오기
- glob 모듈 사용하여 파일 읽기

In [15]:
import glob
import os, re 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
txt_file_path = 'lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r", encoding='utf8') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


### Step 3. 데이터 정제
- 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거
- => 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기

In [3]:
import re

# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(sentence.split(' ')) >= 15: continue  # cf. 토큰 n개 넘어가면 제외
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>',
 '<start> had some bad love <end>']

### Step 4. 평가 데이터셋 분리
- 훈련 데이터와 평가 데이터를 분리
- 단어장의 크기는 12,000 이상, 총 데이터의 20% 를 평가 데이터셋으로 사용

In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 단어장 크기 n # n개 딘어를 기억할 수 있는 tokenizer 생성
        filters=' ',  # 앞에서 전처리를 했기 때문에 filters 설정할 필요 없음
        oov_token="<unk>"  # num_words에 포함되지 못한 단어는 '<unk>'로 변경
    )
    
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)
    # 입력 데이터의 시퀀스 길이를 문장 뒤(padding='post')에 패딩을 붙여 일정하게 맞춰줍니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  # maxlen=15: 토큰 개수 15개 넘어가는 데이터는 학습 데이터에서 제외
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 289  28 ...   0   0   0]
 [  2 229  13 ...   0   0   0]
 [  2  25  17 ...   0   0   0]
 ...
 [  2  22  76 ...   0   0   0]
 [  2  44  26 ...   0   0   0]
 [  2  22  76 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x000002B82D08CCA0>


In [6]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:] 

In [7]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                    tgt_input, 
                                                    test_size=0.2, 
                                                    random_state=7)

In [8]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (126550, 14)
Target Train: (126550, 14)


### Step 5. 인공지능 만들기
- 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)
- 모델이 생성한 가사 한 줄 확인

In [9]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256

 # tokenizer가 구축한 단어사전 내 n개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# -데이터셋 참고: https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
# Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
optimizer = tf.keras.optimizers.Adam()
model.compile(loss=loss, optimizer=optimizer)

# cf. validation_data와 batch_size는 같이 지정해야 오류가 발생하지 않음
model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val), batch_size=BATCH_SIZE)

Epoch 1/10
494/494 [==============================] - 317s 601ms/step - loss: 3.3999 - val_loss: 2.9984
Epoch 2/10
494/494 [==============================] - 297s 600ms/step - loss: 2.8261 - val_loss: 2.7160
Epoch 3/10
494/494 [==============================] - 293s 593ms/step - loss: 2.5096 - val_loss: 2.5296
Epoch 4/10
494/494 [==============================] - 295s 597ms/step - loss: 2.2038 - val_loss: 2.3890
Epoch 5/10
494/494 [==============================] - 284s 575ms/step - loss: 1.9202 - val_loss: 2.2948
Epoch 6/10
494/494 [==============================] - 285s 578ms/step - loss: 1.6686 - val_loss: 2.2298
Epoch 7/10
494/494 [==============================] - 293s 594ms/step - loss: 1.4571 - val_loss: 2.1890
Epoch 8/10
494/494 [==============================] - 292s 590ms/step - loss: 1.2855 - val_loss: 2.1816
Epoch 9/10
494/494 [==============================] - 286s 578ms/step - loss: 1.1575 - val_loss: 2.1868
Epoch 10/10
494/494 [==============================] - 298s 604m

In [16]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  3072256   
                                                                 
 lstm (LSTM)                 multiple                  18882560  
                                                                 
 lstm_1 (LSTM)               multiple                  33562624  
                                                                 
 dense (Dense)               multiple                  24590049  
                                                                 
Total params: 80,107,489
Trainable params: 80,107,489
Non-trainable params: 0
_________________________________________________________________


데이터가 커서 훈련하는 데 시간이 제법 걸릴 겁니다. 여유를 가지고 작업하시면 좋아요 :)

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [50]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [49]:
generate_text(model, tokenizer, init_sentence="<start> i like", max_len=20)

'<start> i like the way how you re holdin me <end> '

---

# 회고

### - 이번 프로젝트에서 **어려웠던 점**.
열심히 50분동안 학습시켰으나 "\<start> i love"를 입력하면 모델이 내뱉는 말은 고작 you 뿐이었다. 단 세 글자 y, o, u를 뱉어냈다. 어렵다기 보다는 그 점이 아쉬웠다.

### - 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**.
1. 정규표현식을 이용하여 텍스트 전처리를 하고 원하지 않는 문장은 넘길 수 있었다.
2. validation_loss 값을 측정하려면 model.fit()의 인자로 validation_data와 batch_size 값을 지정해야 한다.
3. hidden_size 값을 늘리면 학습 시간이 엄청 늘어난다.
4. validation_loss 값을 낮추는 것이 무조건 능사는 아니다.

### - 루브릭 평가 지표를 맞추기 위해 **시도한 것들**.

>#### **루브릭**
>|번호|평가문항|상세기준|
>|:---:|---|---|
>|1|가사 텍스트 생성 모델이 정상적으로 동작하는가?|텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?|
>|2|데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?|특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?|
>|3|텍스트 생성모델이 안정적으로 학습되었는가?|텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?|

validation_loss 값을 2.2 이하로 낮추기 위해 단어장 개수(tokenizer의 num_words)를 늘여보고 줄여보고 해봤지만 별로 효과는 없었다. batch_size 값을 2배로 설정해봤으나 2.2 이하로 낮추는 것에 실패했다. hidden_size 값을 2배로 설정했더니 성공했다. 하지만 그만큼 학습 시간이 많이 늘어났기에 오랜 시간을 기다려야만 했다.

"\<start> i love"를 입력했더니 내뱉는 단어는 "you" 하나였다. 50분 동안 학습을 기다린 것에 비해 매우 비효율적인 결과물이었다. 그래도 "\<start> i like"를 입력하면 좀 더 길게 문장을 만들어 냈다. 루브릭 평가 지표를 맞췄지만 만족할만 한 결과가 나오지 않아 살짝 실망했다.

### - 만약에 루브릭 평가 관련 지표를 **달성 하지 못했을 때, 이유에 관한 추정**.
없음

### - **자기 다짐**
정규표현식 공부가 더 필요하다고 느껴졌다.